### Setup

In [1]:
import os
from icecream import ic
from os.path import join
import pickle
import numpy as np
import open3d as o3d
from dataloader import *
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import mpl_toolkits.mplot3d.art3d as art3d

from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import norm
# import pylab as plt

import warnings
warnings.filterwarnings('ignore')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Utils

In [2]:
def get_angle(u, v):
    t = np.linalg.norm(u) * np.linalg.norm(v)
    return np.abs(np.arccos(np.dot(u, v) / t))


def calculate_distances_to_cone(apex, axis_vector, theta, points):
    U = points - apex
    U /= np.linalg.norm(U, axis=1, keepdims=True)
    v = axis_vector / np.linalg.norm(axis_vector)

    angles = np.abs(np.arccos(np.dot(U, v)))
    angle_errors = np.abs(angles - theta)
    mask = angle_errors > (np.pi / 2)

    distances_to_apex = np.sqrt(np.sum((points - apex)**2, axis=1))
    res = distances_to_apex * np.sin(angle_errors)
    res = mask * distances_to_apex + (1 - mask) * res
    return res


def calculate_ratio(apex, axis_vector, theta, points, eps=1e-3):
    ds = calculate_distances_to_cone(apex, axis_vector, theta, points)
    return np.sum(ds < eps) / ds.shape[0]


def set_size(ax, w, h):
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)


def plot_cone(points, apex, axis_vector, theta, H=10, xlim=(-10, 10), ylim=(-10,-10), zlim=(-10, 10)):
    fig = plt.figure()
    plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.axes.set_xlim3d(xlim)
    ax.axes.set_ylim3d(ylim)
    ax.axes.set_zlim3d(zlim)
    set_size(ax, 5, 5)
    R1 = H * np.tan(theta)

    v = axis_vector * H
    mag = norm(v)
    v = v / mag
    # make some vector not in the same direction as v
    not_v = np.array([1, 1, 0])
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])
    n1 = np.cross(v, not_v)
    n1 /= norm(n1)
    n2 = np.cross(v, n1)
    n = 200
    t = np.linspace(0, mag, n)
    thetas = np.linspace(0, 2 * np.pi, n)
    t, thetas = np.meshgrid(t, thetas)
    R = np.linspace(0, R1, n)
    X, Y, Z = [apex[i] + v[i] * t + R *
               np.sin(thetas) * n1[i] + R * np.cos(thetas) * n2[i] for i in [0, 1, 2]]
    ax.plot_surface(X, Y, Z, color='blue', linewidth=0, antialiased=True, alpha=0.4)
    ax.plot(points[:,0], points[:,1], points[:,2], 'x', color='red', alpha=0.6)


def load_pc(id=None, kind='test'):
    if kind == 'test':
        path = './dataset/ply/test/pointCloud/pointCloud%d.ply' % id
    else:
        path = './dataset/ply/training/pointCloud/pointCloud%d.ply' % id
    pc = o3d.io.read_point_cloud(path)
    pc.estimate_normals()
    pc.normalize_normals()
    return pc


def print_train_gt(id, pc, eps):
    gt = pickle.load(open('./metadata/ground_truth.pkl', 'rb'))
    ps = gt[id]['params']
    print('Ground Truth:')
    print('   apex        ', ps[4:])
    print('   axis        ', ps[1:4])
    print('   theta       ', ps[0])
    print('   ratio       ', calculate_ratio(
        apex = np.array(ps[4:]),
        axis_vector = np.array(ps[1:4]),
        theta = ps[0],
        points = np.asarray(pc.points),
        eps=eps
    ))

### Fit functions

In [3]:
def calculate_cone_params_simple(points, normals):
    # Apex
    B = np.zeros(3)
    B[:] = normals[:,0] * points[:,0] + normals[:,1] * points[:,1] + normals[:,2] * points[:,2]
    try:
        apex = np.linalg.solve(normals, B) 
    except:
        return None, None, None

    # axis normal
    P = [None, None, None]
    for i in range(3):
        P[i] = points[i] - apex
        P[i] /= np.linalg.norm(P[i])
    u = P[1] - P[0]
    v = P[2] - P[0]
    axis_vector = np.cross(u, v)
    axis_vector /= np.linalg.norm(axis_vector)

    # half the aperture - theta
    theta = get_angle(points[0] - apex, axis_vector)
    if theta > np.pi / 2:
        return None, None, None

    return apex, axis_vector, theta


def fit_cone(pc, n_loop=1000, eps=1e-3, min_points_count=1e9, sample_ratio=1.0):
    all_points = np.asarray(pc.points)
    all_normals = np.asarray(pc.normals)
    
    mn = all_points.min(axis=0)
    mx = all_points.max(axis=0)
    L = np.sqrt(np.sum((mx - mn)**2))
    
    
    best_ratio = 0
    # best_mean_dist = np.inf
    best_apex = None
    best_axis_vector = None
    best_theta = None

    if all_points.shape[0] <= min_points_count:
        sampled_points = np.copy(all_points)
    else:
        n_sample = int(max(all_points.shape[0] * sample_ratio, min_points_count))
        indices = random.sample(list(range(all_points.shape[0])), n_sample)
        sampled_points = all_points[indices]

    count = 0
    while count < n_loop:
        # Calculate params
        indices = np.random.choice(all_points.shape[0], 3, replace=False)
        normals = all_normals[indices] # sample three points, normals
        points = all_points[indices]

        # too_close = False
        # for i in range(2):
        #     for j in range(i+1, 3):
        #         d = np.sqrt(np.sum((points[i] - points[j])**2))
        #         if d < 0.1 * L:
        #             too_close = True
        # if too_close:
        #     continue

        apex, axis_vector, theta = calculate_cone_params_simple(points, normals)
        if apex is None:
            continue
        
        count += 1
        ratio = calculate_ratio(apex, axis_vector, theta, sampled_points, eps=eps)
        # mean_dist = calculate_distances_to_cone(apex, axis_vector, theta, sampled_points).mean()
        if best_ratio < ratio:
        # if best_mean_dist > mean_dist:
            best_ratio = ratio
            # best_mean_dist = mean_dist
            best_apex = apex
            best_axis_vector = axis_vector
            best_theta = theta

    return best_apex, best_axis_vector, best_theta, best_ratio

### Demo functions

In [4]:
def demo_testset(id, n_loop=1000, eps=1e-2):
    pc = load_pc(id=id, kind='test')
    points = np.asarray(pc.points)
    xlim = (np.min(points[:,0]) - 2, np.max(points[:,0]) + 2)
    ylim = (np.min(points[:,1]) - 2, np.max(points[:,1]) + 2)
    zlim = (np.min(points[:,2]) - 2, np.max(points[:,2]) + 2)
    H = max([t[1] - t[0] for t in (xlim, ylim, zlim)])
    apex, axis_vector, theta, ratio = fit_cone(pc, n_loop=n_loop, eps=eps)
    plot_cone(points, apex, axis_vector, theta, H=5, xlim=xlim, ylim=ylim, zlim=zlim)

    print('Prediction')
    print('apex        ', apex)
    print('axis_vector ', axis_vector)
    print('theta       ', theta)
    print('ratio       ', ratio)


def demo_trainset(id, eps=1e-2, n_loop=1000):
    pc = load_pc(id=id, kind='train')
    print_train_gt(id, pc, eps)
    points = np.asarray(pc.points)
    xlim = (np.min(points[:,0]) - 2, np.max(points[:,0]) + 2)
    ylim = (np.min(points[:,1]) - 2, np.max(points[:,1]) + 2)
    zlim = (np.min(points[:,2]) - 2, np.max(points[:,2]) + 2)
    H = max([t[1] - t[0] for t in (xlim, ylim, zlim)])
    # apex, axis_vector, theta = fit_cone(pc, n_loop_apex=10000, n_loop_axis=10000)
    apex, axis_vector, theta, ratio = fit_cone(pc, n_loop=n_loop, eps=eps)
    plot_cone(points, apex, axis_vector, theta, H=5, xlim=xlim, ylim=ylim, zlim=zlim)

    print('Prediction')
    print('   apex        ', apex)
    print('   axis_vector ', axis_vector)
    print('   theta       ', theta)
    # print('   ratio       ', ratio)

### Infer

In [7]:
def generate_result(labelpath):
    test_label = pickle.load(open(labelpath, 'rb'))
    cone_ids = [id for id in range(1, 926) if test_label[id] == 4]

    for id in tqdm(cone_ids):
        filepath = f'./result/{id}_4.txt'
        if os.path.exists(filepath):
            continue
        pc = load_pc(id=id, kind='test')
        apex, axis_vector, theta, mean_dist = fit_cone(pc, n_loop=10000)
        with open(filepath, 'w') as f:
            lines = [
                '4\n',
                f'{theta}\n',
                f'{axis_vector[0]}\n',
                f'{axis_vector[1]}\n',
                f'{axis_vector[2]}\n',
                f'{apex[0]}\n',
                f'{apex[1]}\n',
                f'{apex[2]}',
            ]
            f.write(''.join(lines))

In [8]:
generate_result('./res_dict/test_label_runA.pkl')
generate_result('./res_dict/test_label_runB.pkl')

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/173 [00:00<?, ?it/s]